In [439]:
import requests
import json
import pandas as pd
import os
import time 
from IPython.display import clear_output 
from operator import itemgetter
import numpy as np
from pandas.io.json import json_normalize

#set wd
os.chdir("//rca-file-server/RCA/Data Science/Australia Feeds/Company Matching Geocodes")



df = pd.read_excel("company_address2.xlsx")  
df1 = pd.read_excel("this.xlsx")

In [440]:
df.shape
print(type(df['response'][1]))

(12464, 2)

In [442]:
df['response'] = df['response'].str.replace('True','true')
df['response'] = df['response'].str.replace('False','false')
df['response'] = df['response'].str.replace('None','"none"')
df['response'] = df['response'].str.replace("'", '"')
df['response'] = df['response'].replace("\\", r"\\")


In [443]:
type(json.loads(df['response'][1]))

dict

In [447]:
# rows list initialization 
rows = [] 
  
# appending rows 
for data in list: 
    data_row = df['response']  
      
    for row in data_row: 
        row['Name']= json.loads(data_row) 
        rows.append(row) 
  
# using data frame 
df = pd.DataFrame(rows) 
  
# print(df) 

TypeError: 'type' object is not iterable

In [415]:
final = pd.DataFrame.from_dict(pd.json_normalize(df1['response']) ,orient='columns')


In [448]:
json.loads(df['response'][0])

{'address_components': [{'long_name': 'Bundall Road & Slatyer Avenue',
   'short_name': 'Bundall Rd & Slatyer Ave',
   'types': ['intersection']},
  {'long_name': 'Bundall',
   'short_name': 'Bundall',
   'types': ['locality', 'political']},
  {'long_name': 'City of Gold Coast',
   'short_name': 'Gold Coast',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'Queensland',
   'short_name': 'QLD',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'Australia',
   'short_name': 'AU',
   'types': ['country', 'political']},
  {'long_name': '4217', 'short_name': '4217', 'types': ['postal_code']}],
 'formatted_address': 'Bundall Rd & Slatyer Ave, Bundall QLD 4217, Australia',
 'geometry': {'location': {'lat': -27.9976302, 'lng': 153.4141172},
  'location_type': 'GEOMETRIC_CENTER',
  'viewport': {'northeast': {'lat': -27.9962812197085,
    'lng': 153.4154661802915},
   'southwest': {'lat': -27.9989791802915, 'lng': 153.4127682197085}}},
 'partia

In [449]:
df['response'][0]

'{"address_components": [{"long_name": "Bundall Road & Slatyer Avenue", "short_name": "Bundall Rd & Slatyer Ave", "types": ["intersection"]}, {"long_name": "Bundall", "short_name": "Bundall", "types": ["locality", "political"]}, {"long_name": "City of Gold Coast", "short_name": "Gold Coast", "types": ["administrative_area_level_2", "political"]}, {"long_name": "Queensland", "short_name": "QLD", "types": ["administrative_area_level_1", "political"]}, {"long_name": "Australia", "short_name": "AU", "types": ["country", "political"]}, {"long_name": "4217", "short_name": "4217", "types": ["postal_code"]}], "formatted_address": "Bundall Rd & Slatyer Ave, Bundall QLD 4217, Australia", "geometry": {"location": {"lat": -27.9976302, "lng": 153.4141172}, "location_type": "GEOMETRIC_CENTER", "viewport": {"northeast": {"lat": -27.9962812197085, "lng": 153.4154661802915}, "southwest": {"lat": -27.9989791802915, "lng": 153.4127682197085}}}, "partial_match": true, "place_id": "EjVCdW5kYWxsIFJkICYgU2xh